In [ ]:
from deepinv.physics.generator import DiffractionBlurGenerator
from src.utils.display_utils import show_images
import matplotlib.pyplot as plt
from PIL import Image
import torch
import torchvision.transforms as transform
from product_convolution import image_to_patches, patches_to_image, product_convolution2d_patches, get_psf_product_convolution2d_patches, unity_partition_function_2d, crop_unity_partition_2d, compute_patch_info

In [ ]:
image_size = 546
image = torch.randn(1,3,image_size,image_size)
patch_size = 128
overlap = 64

patches = image_to_patches(image, patch_size=patch_size, overlap=overlap)
image_r = patches_to_image(patches, overlap)

In [ ]:
print(patches.shape)
print(image_r.shape)
print(compute_patch_info(image.shape[-2:], patch_size, overlap))

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib import gridspec
from torch import Tensor
import numpy as np
from typing import List, Optional, Tuple, Union
import warnings
warnings.filterwarnings("ignore", module="matplotlib\..*")

_TensorArray = Union[Tensor, np.ndarray]

def to_numpy_image(input):
    if isinstance(input, Tensor):
        if input.dim() == 3:
            return input.detach().cpu().permute(1, 2, 0).numpy()
        elif input.dim() == 4:
            return input.detach().cpu().permute(0, 2, 3, 1).numpy()

    elif isinstance(input, np.ndarray):
        print("Warning: input is already a numpy array")
        return input
    else:
        raise ValueError(f"Cannot convert {type(input)} to numpy image")
def show_images(
    imgs: Union[_TensorArray, List[_TensorArray], Tuple[_TensorArray]],
    title: List[str] = None,
    suptitle: str = None,
    ncols: int = None,
    colorbar: Optional[bool] = False,
    cmap: str = None,
    vmin: Optional[float] = None,
    vmax: Optional[float] = None,
    savename: Optional[str] = None,
    figsize: Optional[int] = 3,
    interpolation: Optional[str] = None,
):

    if isinstance(imgs, List) or isinstance(imgs, Tuple):
        if isinstance(imgs[0], np.ndarray):
            imgs = np.concatenate(imgs, axis=0)
        elif isinstance(imgs[0], torch.Tensor):
            imgs = to_numpy_image(torch.cat(imgs, dim=0))
    else:
        imgs = to_numpy_image(imgs)

    if imgs.ndim == 3:
        H, W, C = imgs.shape
        B = 1
    elif imgs.ndim == 4:
        B, H, W, C = imgs.shape

    if (ncols is not None) and (B % ncols == 0):
        nrows = B // ncols
    else:
        nrows = 1
        ncols = B
    offset = 0 if not colorbar else 0.75
    fig = plt.figure(figsize=(ncols * (figsize + offset), nrows * figsize))
    gs = gridspec.GridSpec(ncols=ncols, nrows=nrows)
    gs.update(wspace=0.025, hspace=0.025)

    if cmap is None:
        cmap = "gray" if C == 1 else None
    i = 0

    if title is None:
        title = [""]
    if len(title) != B:
        title += [""] * (B - len(title))
    with warnings.catch_warnings():
        if nrows > 1 and ncols > 1:
            for r in range(nrows):
                for c in range(ncols):
                    axs = plt.subplot(gs[r, c])
                    im = axs.imshow(
                        imgs[i],
                        vmin=vmin,
                        vmax=vmax,
                        cmap=cmap,
                        interpolation=interpolation,
                    )
                    axs.set_xticks([])
                    axs.set_yticks([])
                    if colorbar:
                        divider = make_axes_locatable(axs)
                        cax = divider.append_axes("right", size="3%", pad=0.1)
                        plt.colorbar(im, cax=cax)
                    if title:
                        axs.set_title(title[i], pad=3)

                    i += 1
        elif nrows > 1 or ncols > 1:
            for c in range(max(ncols, nrows)):
                axs = plt.subplot(gs[c])
                im = axs.imshow(
                    imgs[c], vmin=vmin, vmax=vmax, cmap=cmap, interpolation=interpolation
                )
                axs.set_xticks([])
                axs.set_yticks([])
                if colorbar:
                    divider = make_axes_locatable(axs)
                    cax = divider.append_axes("right", size="3%", pad=0.1)
                    plt.colorbar(im, cax=cax)
                if title:
                    axs.set_title(title[c], pad=3)

        else:
            axs = plt.subplot(gs[0])
            im = axs.imshow(
                imgs.squeeze(), vmin=vmin, vmax=vmax, cmap=cmap, interpolation=interpolation
            )
            axs.set_xticks([])
            axs.set_yticks([])
            if colorbar:
                divider = make_axes_locatable(axs)
                cax = divider.append_axes("right", size="3%", pad=0.1)
                plt.colorbar(im, cax=cax)
            if title:
                axs.set_title(title[0], pad=3)

        if suptitle is not None:
            fig.suptitle(suptitle, y=1.03)
        if savename is not None:
            fig.savefig(savename, bbox_inches='tight')
            plt.close()
        else:
            plt.show()

In [ ]:
# %% TEST PRODUCT-CONVOLUTION
path_1 = '/home/minhhai/Works/datasets/FFHQ/images1024x1024/00003.png'
path_2 = '/home/minhhai/Works/datasets/FFHQ/images1024x1024/00014.png'

image_1 = Image.open(path_1)
image_1 = transform.ToTensor()(image_1)

image_2 = Image.open(path_2)
image_2 = transform.ToTensor()(image_2)

image = torch.stack([image_1, image_2], dim=0).cuda()
show_images(image)

In [ ]:
psf_size = (31, 31)
patch_size = (128, 128)
overlap = (64, 64)
image_size = (512,512)
image = torch.zeros((1,3) + image_size).cuda()
image[...,30,30] = 1.
patches = image_to_patches(image, patch_size, overlap)
patch_info = compute_patch_info(image_size, patch_size, overlap)
K = patches.size(2) * patches.size(3)


generator = DiffractionBlurGenerator(
    psf_size, num_channels=3, fc=0.2, device='cuda')
h = torch.stack([generator.step(image.size(0))['filter']
                 for _ in range(K)], dim=2)
masks = unity_partition_function_2d(image.shape[-2:], patch_size, overlap)
w, _ = crop_unity_partition_2d(masks, patch_size, overlap, psf_size)
w = w.unsqueeze(0).unsqueeze(1).flatten(2,3).cuda()

show_images(h.transpose(2,1).flatten(0, 1) ** 0.25, ncols=K)
blurry = product_convolution2d_patches(image, w, h, patch_size, overlap)
show_images(blurry ** 0.25)

In [ ]:
position = (45, 68)
psf_size = (31, 31)
patch_size = (128, 128)
overlap = (64, 64)
image_size = 256

image = torch.zeros(2, 3, image_size, image_size, device='cuda')
image[..., position[0]: position[0] + 1, position[1]: position[1] + 1] = 1.
show_images(image)
info = compute_patch_info(image_size, patch_size, overlap)
K = np.prod(info['num_patches'])
masks = unity_partition_function_2d(image.shape[-2:], patch_size, overlap)
w, _ = crop_unity_partition_2d(masks, patch_size, overlap, psf_size)
w = w.unsqueeze(0).unsqueeze(1).flatten(2,3).cuda()
# w = torch.ones_like(w)
h = torch.stack([generator.step(image.size(0))['filter']
                 for _ in range(K)], dim=2)
blurry = product_convolution2d_patches(image, w, h, patch_size, overlap)

psf_pc = blurry[..., position[0] - psf_size[0] // 2: position[0] + psf_size[0] //
            2 + 1, position[1] - psf_size[1] // 2: position[1] + psf_size[1] // 2 + 1]

patches = image_to_patches(image, patch_size, overlap)
psf = get_psf_product_convolution2d_patches(
    h, w, position, overlap, patches.shape[2:4])

diff = psf_pc[:,0:1,...] - psf[:,0:1,...]
show_images(torch.cat((psf_pc[:,0:1,...], psf[:,0:1,...], diff), dim = 0) ** 0.25, colorbar=True, suptitle=f'Norm Diff: {torch.norm(diff).item():.5f}')


In [ ]:
position = (45, 0)
psf_size = (31, 31)
patch_size = (128, 128)
overlap = (64, 64)

image = torch.zeros(2, 3, 256, 256, device='cuda')
image[..., position[0]: position[0] + 1, position[1]: position[1] + 1] = 1.
show_images(image)

masks = unity_partition_function_2d(image.shape[-2:], patch_size, overlap)
w, _ = crop_unity_partition_2d(masks, patch_size, overlap, psf_size)
w = w.unsqueeze(0).unsqueeze(1).flatten(2,3).cuda()
# w = torch.ones_like(w)
h = torch.stack([generator.step(image.size(0))['filter']
                 for _ in range(K)], dim=2)
blurry = product_convolution2d_patches(image, w, h, patch_size, overlap)

psf_pc = blurry[..., :psf_size[0],: psf_size[1]]

patches = image_to_patches(image, patch_size, overlap)
psf = get_psf_product_convolution2d_patches(
    h, w, position, overlap, patches.shape[2:4])

diff = psf_pc[:,0:1,...] - psf[:,0:1,...]
show_images(torch.cat((psf_pc[:,0:1,...], psf[:,0:1,...], diff), dim = 0) ** 0.25, colorbar=True, suptitle=f'Norm Diff: {torch.norm(diff).item():.5f}')

In [ ]:
psf_size = (31, 31)
patch_size = (128, 128)
overlap = (64, 64)
image = torch.zeros(2, 3, 256, 256, device='cuda')

masks = unity_partition_function_2d(image.shape[-2:], patch_size, overlap)
w, _ = crop_unity_partition_2d(masks, patch_size, overlap, psf_size)
w = w.unsqueeze(0).unsqueeze(1).flatten(2,3).cuda()
# w = torch.ones_like(w)
h = torch.stack([generator.step(image.size(0))['filter']
                 for _ in range(K)], dim=2)

for i in range(psf_size[0] // 2, 256 - psf_size[0] // 2):
    for j in range(psf_size[1] // 2, 256 - psf_size[1] // 2):
        position = (i, j)
        image = torch.zeros(2, 3, 256, 256, device='cuda')
        image[..., position[0]: position[0] + 1, position[1]: position[1] + 1] = 1.
        blurry = product_convolution2d_patches(image, w, h, patch_size, overlap)

        psf_pc = blurry[..., position[0] - psf_size[0] // 2: position[0] + psf_size[0] //
                    2 + 1, position[1] - psf_size[1] // 2: position[1] + psf_size[1] // 2 + 1]
        patches = image_to_patches(image, patch_size, overlap)
        psf = get_psf_product_convolution2d_patches(
            h, w, position, overlap, patches.shape[2:4])

        diff = psf_pc[:,0:1,...] - psf[:,0:1,...]
        if torch.norm(diff) > 1e-5:
            print(f'{i, j}: ', torch.norm(diff))